In [ ]:
import os
import sys
import pandas as pd

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

max_drawdown_global = 0
max_drawdown_controller = 0
hb_image = "dardonacci/hummingbot:latest"
credentials = "master_account"
controller_id_list = ["94et2aiooxBhYQ9iLMPmsNABjV5b2hCQo1ZTo9XzLv6w", "3Dgvw7vgZX4Hjdea3oqWuFQEpUERwAUpiYyxAQk5z5Ey"]
bot_name = "macd_mt_28_nov"
days_to_cash_out = 3
deployment_host = "localhost"


In [28]:
from core.backtesting.optimizer import StrategyOptimizer
from core.services.backend_api_client import BackendAPIClient

import json
from core.utils import dump_dict_to_yaml



def preprocess_trials_df(trials_df):
    df = trials_df.copy()
    df.columns = [col.replace("params_", "") if col.startswith("params_") else col for col in df.columns]
    df["executors"] = df["executors"].apply(lambda x: json.loads(x))
    df["config"] = df["config"].apply(lambda x: json.loads(x))
    df["trading_pair"] = df["config"].apply(lambda x: x['trading_pair'])
    df["from_timestamp"] = df["executors"].apply(
        lambda x: pd.to_datetime(pd.Series(x["timestamp"].values()).min(), unit="s"))
    df["to_timestamp"] = df["executors"].apply(
        lambda x: pd.to_datetime(pd.Series(x["close_timestamp"].values()).max(), unit="s"))
    return df


optimizer = StrategyOptimizer(root_path=root_path)
client = BackendAPIClient(
    host=deployment_host,
    port=8000,
)

all_studies = optimizer.get_all_study_names()
dfs = []
for study_name in all_studies:
    trials_df = optimizer.get_study_trials_df(study_name)
    trials_df["study_name"] = study_name
    dfs.append(trials_df)

trials = pd.concat(dfs)
trials = preprocess_trials_df(trials)
trials['from_timestamp'] = pd.to_datetime(trials['from_timestamp'])
trials['to_timestamp'] = pd.to_datetime(trials['to_timestamp'])
trials['days_diff'] = (trials['to_timestamp'] - trials['from_timestamp']).dt.total_seconds() / 86400
trials['executors_per_day'] = trials['total_executors'] / trials['days_diff']
trials['volume_per_day'] = trials['total_volume'] / trials['days_diff']
trials.drop_duplicates(subset=["config"], inplace=True)
configs = trials["config"]
filtered_configs = configs[configs.apply(lambda x: x['id'] in controller_id_list)]
print(len(filtered_configs))
print(filtered_configs)


2
18    {'id': '3Dgvw7vgZX4Hjdea3oqWuFQEpUERwAUpiYyxAQ...
21    {'id': '94et2aiooxBhYQ9iLMPmsNABjV5b2hCQo1ZTo9...
Name: config, dtype: object


In [29]:
config_list = filtered_configs.tolist()
for config in config_list:
    await client.add_controller_config(config)
config_ids = [config["id"] + ".yml" for config in config_list]

await client.deploy_script_with_controllers(
        bot_name=bot_name,
        controller_configs=config_ids,
        image_name=hb_image,
        credentials=credentials,
        time_to_cash_out=days_to_cash_out*86400,
        max_global_drawdown=max_drawdown_global,
        max_controller_drawdown=max_drawdown_controller,
    )